# **MULTIPLICACIÓN DE MATRICES**

In [23]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pb4e_egd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pb4e_egd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [24]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin




---

# Multiplicacion clasica y de stressen secuencial

Sin punteros

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int NUM = 3;
float a = 1.0;

int main(void){
  clock_t start, end;
  float A[NUM][NUM];
  float B[NUM][NUM];
  float C[NUM][NUM];

  for(int i = 0; i < NUM; i ++){
      for(int j = 0; j < NUM; j++){
          A[i][j] = (float)rand()/(float)(RAND_MAX/a);
          B[i][j] = (float)rand()/(float)(RAND_MAX/a);
      }
  }

  // classic multiplication
  start = clock();

  for(int i = 0; i < NUM; i ++){
      for(int j = 0; j < NUM; j++){
        C[i][j] = 0;
        for(int k = 0; k < NUM; k++){
            C[i][j] += A[i][k] * B[k][j];
        }
      }
  }

  printf("A: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", A[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("B: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", B[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("C: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", C[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  end=clock();
  double cpu_time_used = ((double) (end-start))/ CLOCKS_PER_SEC;
  printf("classic time: %f",cpu_time_used);
}

A: 
0.840188 0.783099 0.911647 
0.335223 0.277775 0.477397 
0.364784 0.952230 0.635712 
B: 
0.394383 0.798440 0.197551 
0.768230 0.553970 0.628871 
0.513401 0.916195 0.717297 
C: 
1.400996 1.939900 1.312371 
0.590697 0.858923 0.583344 
1.201771 1.401201 1.126887 
classic time: 0.000025


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int NUM = 2;
float a = 5.0;


int main(void){
  clock_t start, end;
  int A[NUM][NUM];
  int B[NUM][NUM];
  int C[NUM][NUM];

  for(int i = 0; i < NUM; i ++){
      for(int j = 0; j < NUM; j++){
          A[i][j] = (float)rand()/(float)(RAND_MAX/a);
          B[i][j] = (float)rand()/(float)(RAND_MAX/a);
      }
  }
  //printf();

  // stressen multiplication
  start = clock();

  // create matrixs M1 - M7 with sizes NUM/2 * NUM/2

  int M1[NUM/2][NUM/2];
  int M2[NUM/2][NUM/2];
  int M3[NUM/2][NUM/2];
  int M4[NUM/2][NUM/2];
  int M5[NUM/2][NUM/2];
  int M6[NUM/2][NUM/2];
  int M7[NUM/2][NUM/2];

  // fill matrix acording stressen algorithm 

  for(int i = 0; i < NUM/2; i ++){
      for(int j = 0; j < NUM/2; j++){
        M1[i][j] = (A[i][j] + A[NUM/2 + i][NUM/2 + j]) * (B[i][j] = B[NUM/2 + i][NUM/2 + j]);
        M2[i][j] = (A[NUM/2 + i][j] + A[NUM/2 + i][NUM/2 + j]) * B[i][j];
        M3[i][j] = A[i][j] * (B[i][j] + B[NUM/2 + i][NUM/2 + j]);
        M4[i][j] = A[NUM/2 + i][NUM/2 + j] * (B[NUM/2 + i][i] - B[i][j]);
        M5[i][j] = (A[i][j] + A[i][NUM/2 + j]) * B[NUM/2 + i][NUM/2 + j];
        M6[i][j] = (A[NUM/2 + i][j] - A[i][j]) * (B[i][j] + B[i][NUM/2 + j]);
        M7[i][j] = (A[i][j] - A[NUM/2 + i][NUM/2 + j]) * (B[NUM/2 + i][j] + B[NUM/2 + i][NUM/2 + j]);
      }
  }

  for(int i = 0; i < NUM/2; i++) {
      for(int j = 0; j < NUM/2; j++) {
          C[i][j] = M1[i][j] + M4[i][j] - M5[i][j] + M7[i][j];
          C[i][NUM/2 + j] = M3[i][j] + M5[i][j];
          C[NUM/2 + i][j] = M2[i][j] + M4[i][j];
          C[NUM/2 + i][NUM/2 + j] = M1[i][j] - M2[i][j] + M3[i][j] + M6[i][j];
      }
  }
  end=clock();
  double cpu_time_used = ((double) (end-start))/ CLOCKS_PER_SEC;
  printf("stressen time: %f",cpu_time_used);

  printf("A: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", A[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("B: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", B[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M1: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M1[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M2: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M2[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M2: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M2[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M3: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M3[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M4: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M4[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M5: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M5[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M6: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M6[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("M7: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", M7[i][j]);
          printf(" ");
      }
      printf("\n");
  }

  printf("C: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f", C[i][j]);
          printf(" ");
      }
      printf("\n");
  }

}

stressen time: 0.000001A: 
0.000001 0.000001 
0.000001 0.000001 
B: 
0.000001 0.000001 
0.000001 0.000001 
M1: 
0.000001 0.000001 
0.000001 0.000001 
M2: 
0.000001 0.000001 
0.000001 0.000001 
M2: 
0.000001 0.000001 
0.000001 0.000001 
M3: 
0.000001 0.000001 
0.000001 0.000001 
M4: 
0.000001 0.000001 
0.000001 0.000001 
M5: 
0.000001 0.000001 
0.000001 0.000001 
M6: 
0.000001 0.000001 
0.000001 0.000001 
M7: 
0.000001 0.000001 
0.000001 0.000001 
C: 
0.000001 0.000001 
0.000001 0.000001 



Con punteros

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int NUM = 1024;

int main(void){
  clock_t start, end;
  start = clock();

  float *A = (float *)malloc(NUM * NUM * sizeof(float));
  float *B = (float *)malloc(NUM * NUM * sizeof(float));
  float *C = (float *)malloc(NUM * NUM * sizeof(float));
  float a = 1.0;

  // Matrix A and B have random values
  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          *(A + i * NUM + j) = (float)rand()/(float)(RAND_MAX/a);
          *(B + i * NUM + j) = (float)rand()/(float)(RAND_MAX/a);
      }
  }

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
        *(C + i * NUM + j) = 0;
        for (int k = 0; k < NUM; k++) {
            *(C + i * NUM + j) += (*(A + i * NUM + k)) * (*(B + k * NUM + j));
        }
      }
  }

  /*printf("A: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(A + i * NUM + j));
      }
      printf("\n");
  }

  printf("B: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(B + i * NUM + j));
      }
      printf("\n");
  }

  printf("C: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(C + i * NUM + j));
      }
      printf("\n");
  }*/

  end=clock();
  double cpu_time_used = ((double) (end-start))/ CLOCKS_PER_SEC;
  printf("%f",cpu_time_used);

  free(A);
  free(B);
  return 0;
}

16.947889




---

# Multiplicacion con OpenMP

32 64 128 256 512 1024 matrices
2 4 8 16 32 hilos

In [ ]:
!gcc -o omps -fopenmp openMPSec.c

In [ ]:
!./omps

Time elapsed: 7.820039

In [ ]:
#include <stdlib.h>
#include <stdio.h>
#include <sys/time.h>
#include <omp.h>

#define NUM 1024
#define NUM_THREADS 32

struct timeval tv; 
double get_clock() {
   struct timeval tv; int ok;
   ok = gettimeofday(&tv, (void *) 0);
   return (tv.tv_sec * 1.0 + tv.tv_usec * 1.0E-6); 
}

void check_mul (double *A, double *B, double *C, int n) {
    int i, j, k, sum;
    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
           sum = 0;
           for (k = 0; k < n; k++) {
               sum += A[i * n + k] * B[k * n + j];
           }
           if (sum != C[i * n + j]) {
               printf("Error multiplying matrices\n");
               exit(1);
           }
        }
    }
    printf("Multiplication ok\n");
}

int main(int argc, char *argv[]) {

	int i,j,k,ii,jj,kk,sum;
	double start,end;
	int numthreads,tid;

    double *A = (double *)malloc(NUM * NUM * sizeof(double));
    double *B = (double *)malloc(NUM * NUM * sizeof(double));
    double *C = (double *)malloc(NUM * NUM * sizeof(double));

    // Matrix A and B have random values
    for (int i = 0; i < NUM; i++) {
        for (int j = 0; j < NUM; j++) {
            *(C + i * NUM + j) = 0;
            *(A + i * NUM + j) = (float)rand()/(float)(RAND_MAX);
            *(B + i * NUM + j) = (float)rand()/(float)(RAND_MAX);
        }
    }

    omp_set_num_threads(NUM_THREADS);

    #pragma omp parallel
    {
        if(omp_get_thread_num()==0)
                printf("Running with %d threads\n",omp_get_num_threads());
    }

	start = get_clock();
	
	#pragma omp parallel for private(i,j,k) 
    for (int i = 0; i < NUM; i++) {
        for (int j = 0; j < NUM; j++) {
            *(C + i * NUM + j) = 0;
            for (int k = 0; k < NUM; k++) {
                *(C + i * NUM + j) += (*(A + i * NUM + k)) * (*(B + k * NUM + j));
        }
      }
    }
	
	end = get_clock();
	printf("Time: %lf\n",(end-start));

    //check_mul(A,B,C,NUM);

    free(A);
    free(B);
    free(C);
	return 0;
}



---
# CUDA


In [71]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [72]:
%cd /usr/local/cuda-11/samples/1_Utilities/deviceQuery
!make
!./deviceQuery

[Errno 2] No such file or directory: '/usr/local/cuda-11/samples/1_Utilities/deviceQuery'
/content
make: *** No targets specified and no makefile found.  Stop.
/bin/bash: ./deviceQuery: No such file or directory


Paralelización Personalizada

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int NUM = 1024;

__global__ void matrixMult(float *a, float *b, float *c, int n, int block_size, int grid_size){
    
  int index = (blockIdx.x * blockDim.x) + threadIdx.x;

  int inicio = index * (n*n)/(block_size*grid_size);
  int fin = (index + 1) * (n*n)/(block_size*grid_size);

  for(long idx = inicio; idx < fin; idx++){
        int i = idx / n;
        int j = idx % n;
        float sum = 0;
        for(int k = 0; k < n; k++){
            sum += *(a + (i * n) + k) * *(b + (n * k) + j);
        }
        *(c + (i * n) + j) = sum;
  } 

}

int main(void){
  cudaError_t cudaStatus;
  clock_t start, end;

  float *A = (float*)malloc(NUM * NUM * sizeof(float));
  float *B = (float*)malloc(NUM * NUM * sizeof(float));
  float *C = (float*)malloc(NUM * NUM * sizeof(float));

  float *d_A, *d_B, *d_C;

  cudaMalloc(&d_A, NUM * NUM * sizeof(float));
  cudaMalloc(&d_B, NUM * NUM * sizeof(float));
  cudaMalloc(&d_C, NUM * NUM * sizeof(float));

  // Matrix A and B have random values
  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          *(A + i * NUM + j) = (float)rand()/(float)(RAND_MAX);
          *(B + i * NUM + j) = (float)rand()/(float)(RAND_MAX);
      }
  }

  start = clock();

  cudaMemcpy(d_A, A, NUM * NUM * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, B, NUM * NUM * sizeof(float), cudaMemcpyHostToDevice);

  //int space = NUM * NUM;
  int blockSize = 128; // 32, 64, 128

  int gridSize = 40; // 1, 20, 40

  matrixMult<<<gridSize,blockSize>>>(d_A, d_B, d_C, NUM, blockSize, gridSize);
  cudaDeviceSynchronize();

  cudaMemcpy(C, d_C, NUM * NUM * sizeof(float), cudaMemcpyDeviceToHost);

  end=clock();

  cudaStatus = cudaGetLastError();
  if (cudaStatus != cudaSuccess) {
     fprintf(stderr, "addKernel launch failed: %s\n", cudaGetErrorString(cudaStatus));
     cudaFree(d_A);
     cudaFree(d_B);
     cudaFree(d_C);
     exit(1);
  }

/*
  printf("A: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(A + i * NUM + j)); // [ 1 2 3 4 5 6 7 8 9]
      }
      printf("\n");
  }

  printf("B: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(B + i * NUM + j));
      }
      printf("\n");
  }


  printf("C: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(C + i * NUM + j));
      }
      printf("\n");
  }
*/
  double cpu_time_used = ((double) (end-start))/ CLOCKS_PER_SEC;
  printf("%f",cpu_time_used);

  return 0;
}

0.034436


Tilling multiplication

In [79]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int NUM = 32;
const int TILE = 32;

__global__ void matrixMult(float *a, float *b, float *c, int n){
  int row = blockIdx.y * TILE + threadIdx.y;
  int col = blockIdx.x * TILE + threadIdx.x;
  float sum = 0;
  for (int i = 0; i < n; i++) {
    __shared__ float s_a[TILE][TILE];
    __shared__ float s_b[TILE][TILE];
    s_a[threadIdx.y][threadIdx.x] = a[row * n + i + threadIdx.x];
    s_b[threadIdx.y][threadIdx.x] = b[(i + threadIdx.y) * n + col];
    __syncthreads();
    for (int j = 0; j < TILE; j++) {
      sum += s_a[threadIdx.y][j] * s_b[j][threadIdx.x];
    }
    __syncthreads();
  }
  c[row * n + col] = sum;
}

int main(void){
  cudaError_t cudaStatus;
  clock_t start, end;
  

  float *A = (float *)malloc(NUM * NUM * sizeof(float));
  float *B = (float *)malloc(NUM * NUM * sizeof(float));
  float *C = (float *)malloc(NUM * NUM * sizeof(float));

  float *d_A, *d_B, *d_C;

  cudaMalloc(&d_A, NUM * NUM * sizeof(float));
  cudaMalloc(&d_B, NUM * NUM * sizeof(float));
  cudaMalloc(&d_C, NUM * NUM * sizeof(float));

  // Matrix A and B have random values
  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          *(A + i * NUM + j) = (float)rand()/(float)(RAND_MAX);
          *(B + i * NUM + j) = (float)rand()/(float)(RAND_MAX);
      }
  }

  start = clock();

  cudaMemcpy(d_A, A, NUM * NUM * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, B, NUM * NUM * sizeof(float), cudaMemcpyHostToDevice);

  dim3 block_size(TILE, TILE,1); // 32, 64, 128

  dim3 grid_size(NUM/TILE,NUM/TILE,1); // 1, 20, 40

    // Use dim3 structs for block  and grid dimensions

  matrixMult<<<grid_size,block_size>>>(d_A, d_B, d_C, NUM);

  cudaMemcpy(C, d_C, NUM * NUM * sizeof(float), cudaMemcpyDeviceToHost);

  end=clock();

  cudaStatus = cudaGetLastError();
  if (cudaStatus != cudaSuccess) {
     fprintf(stderr, "addKernel launch failed: %s\n", cudaGetErrorString(cudaStatus));
     cudaFree(d_A);
     cudaFree(d_B);
     cudaFree(d_C);
     exit(1);
  }
/*
  printf("A: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(A + i * NUM + j));
      }
      printf("\n");
  }

  printf("B: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(B + i * NUM + j));
      }
      printf("\n");
  }

  printf("C: \n");

  for (int i = 0; i < NUM; i++) {
      for (int j = 0; j < NUM; j++) {
          printf("%f ", *(C + i * NUM + j));
      }
      printf("\n");
  }
*/
  double cpu_time_used = ((double) (end-start))/ CLOCKS_PER_SEC;
  printf("%f",cpu_time_used);

  return 0;
}

0.000229


Fuentes:
- Stressen: https://es.wikipedia.org/wiki/Algoritmo_de_Strassen
- http://dis.um.es/~domingo/apuntes/AlgProPar/1718/DV+PD.pdf
- https://mathworld.wolfram.com/BlockMatrix.html#:~:text=When%20two%20block%20matrices%20have,For%20example%2C&text=Note%20that%20the%20usual%20rules,that%20the%20block%20sizes%20correspond).
- https://www.repositoriodigital.ipn.mx/bitstream/123456789/5961/1/PACHUCA%20HERRERA%20YANIRA%20Tesis%202008.pdf
- https://stackoverflow.com/questions/2128728/allocate-matrix-in-c